In [1]:
# DIRECTORY SET
import os
import sys
from pathlib import Path
base_dir=Path(os.getcwd()).parent
os.chdir(base_dir)
print(os.getcwd())

# Load dotenv
import dotenv
dotenv.load_dotenv()

# DJANGO SETUP
import django
sys.path.append(os.path.abspath(''))
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "serverproject.settings")
django.setup()

# Import async modules
import asyncio
from asgiref.sync import sync_to_async

# Import display modules
from IPython.display import display, Markdown

# Import other modules
import faiss

# import reloading
from importlib import reload

d:\DestinyFolder\DestinyRecaps\DestinyRecapsApi\serverproject


In [2]:
%load_ext autoreload
%autoreload 2

from destinyapp.models import StreamRecapData

from core import services
from core import utils

In [ ]:
video_id="n4L4Z0Kx004"
cost=0.0

In [ ]:
stream_recap_data=await utils.get_recap_data(video_id)

In [ ]:
annotated_results, major_topics, minor_topics, temp_cost = await services.stream_plot.generate_data(stream_recap_data)

In [ ]:
system_prompt="""Take the annotated segments from the user and break down the segments into a few major topics and minor topics. Format the response as follows:

Main Topics:
Category: 'Category' | Context: context
...

Minor topics:
- 'minor topic'
...

"""

user_prompt="Here are all the annotationed segments:\n"+annotations_segemnts_str

prompt=[{"role":"system", "content":system_prompt}, {"role":"user", "content":user_prompt}]

In [ ]:
annotations_segemnts_str=""

for i, annoation_result in enumerate(annotated_results):
    annotations_segemnts_str+="Segment: "+str(i)+": "+annoation_result["annotation"]+"\n"
len(annotations_segemnts_str)

In [ ]:
topic_annotations_str2, cost=await utils.async_response_handler(
    prompt,
    utils.ModelNameEnum.claude_3_5_sonnet
)

In [ ]:
print(topic_annotations_str2)


In [ ]:
plot_object, annotated_results, plot_segments, category_locations = await services.stream_plot.process_data(stream_recap_data,  annotated_results, major_topics, minor_topics, video_id)

In [ ]:
plot_object, temp_cost=await services.stream_plot.annotate_extra(video_id, stream_recap_data, plot_object)  

# Data Gen Variant part

In [3]:
from core.services.stream_plot import data_gen
video_id="n4L4Z0Kx004"
stream_recap_data=await utils.get_recap_data(video_id)

In [4]:
cost=0
text_chunks_no_overlap = await data_gen.create_text_chunks(stream_recap_data.transcript, 0)

text_chunk_batches = await data_gen.generate_text_chunk_batches(text_chunks_no_overlap)

topic_annotations_str, temp_cost = await data_gen.annotate_major_minor_topics(stream_recap_data.recap)
cost+=temp_cost
major_topics, minor_topics = data_gen.process_topic_annotations_str(topic_annotations_str)

responses, annotated_results, temp_cost = await data_gen.annotate_all_batches_no_recap(text_chunk_batches)
cost+=temp_cost

Cost:  0.004821000000000001
Cost:  0.021945
Cost:  0.027003
Cost:  0.027963000000000002
Cost:  0.028983000000000002
Cost:  0.030279
Cost:  0.030786
Cost:  0.028053
Cost:  0.030447000000000002
Results:  20
Results:  20
Results:  20
Results:  20
Results:  20
Results:  20
Results:  20
Results:  16


In [5]:
annotated_results_str=""
for ar in annotated_results:
    annotated_results_str+=ar["category"]+": "+ar["annotation"]+"\n"

annotations_segments_str=""
for i, annotation_result in enumerate(annotated_results):
    annotations_segments_str+="Segment: "+str(i)+": "+annotation_result["annotation"]+"\n"

In [6]:
system_prompt="""Take the annotated segments from the user and break down the segments into a few major topics and minor topics. These should be specific and explicit. Format the response as follows:

Main Topics:
Category: 'Category' | Context: context
...

Minor topics:
- 'minor topic'
...

The context should be 1 detailed sentence.
"""

user_prompt="Here are all the annotationed segments:\n"+annotated_results_str#annotations_segments_str

prompt=[{"role":"system", "content":system_prompt}, {"role":"user", "content":user_prompt}]

topic_annotations_str, temp_cost=await utils.async_response_handler(
    prompt,
    utils.ModelNameEnum.claude_3_5_sonnet
)

In [ ]:
system_prompt="""Take the annotated segments from the user and break down the segments into a few major topics and minor topics. These should be specific and explicit. Format the response as follows:

Main Topics:
Category: 'Category' | Context: context
...

Minor topics:
- 'minor topic'
...

The context should be 1 detailed sentence.
"""

user_prompt="Here are all the annotationed segments:\n"+annotated_results_str#annotations_segments_str

prompt=[{"role":"system", "content":system_prompt}, {"role":"user", "content":user_prompt}]

topic_annotations_str, temp_cost=await utils.async_response_handler(
    prompt,
    utils.ModelNameEnum.claude_3_5_sonnet
)

In [7]:
print(topic_annotations_str)

Main Topics:
Category: Mental Health | Context: Dr. Alok Kanojia, a psychiatrist, discusses various mental health topics including stress management, ADHD, work-life balance, and self-improvement strategies.

Category: Personal Development | Context: The conversation explores topics such as decision-making, self-worth, motivation, and developing self-compassion through various psychological and practical approaches.

Category: Neuroscience | Context: The discussion delves into brain health, the impact of lifestyle on cognitive function, and the neurological basis of emotions and social connections.

Category: Work-Life Balance | Context: The conversation addresses the challenges of managing stress, workaholic tendencies, and finding balance between productivity and relaxation, especially for entrepreneurs and individuals with ADHD.

Minor topics:
- Crisis support resources
- Humor and social dynamics
- Parental influence on behavior
- Genetic factors in personality traits
- Breathing e

In [8]:
print(cost)

0.23028


In [9]:
temp_annotation_list=topic_annotations_str.split("Category: ")
major_topics=[]
for temp_annotation in temp_annotation_list:
    if " | Context:" in temp_annotation:
        major_topic=temp_annotation.split(" | Context:")[0].strip()
        # get rid of ' if it starts and ends with that
        if major_topic[0]=="'" and major_topic[-1]=="'":
            major_topic=major_topic[1:-1]

        major_topics.append(major_topic)

minor_topics=[]
if "Minor Topics:" in topic_annotations_str:
    temp_minor_topics=topic_annotations_str.split("Minor Topics:\n")[-1].split("\n- ")
elif "Minor topics:" in topic_annotations_str:
    temp_minor_topics=topic_annotations_str.split("Minor topics:\n")[-1].split("\n- ")
for i in range(len(temp_minor_topics)):
    minor_topic=temp_minor_topics[i].strip()
    if minor_topic!="":
        minor_topics.append(minor_topic)

print(major_topics)
print(minor_topics)

['Mental Health', 'Personal Development', 'Neuroscience', 'Work-Life Balance']
['- Crisis support resources', 'Humor and social dynamics', 'Parental influence on behavior', 'Genetic factors in personality traits', 'Breathing exercises and meditation techniques', 'Laziness and productivity', 'Evolutionary perspectives on behavior', 'Emotional regulation', 'Transcendent compassion', 'ADHD management strategies', 'Social belonging and identity', 'Benefits of solitude']


In [10]:
# run it back with new topics
text_chunks_no_overlap = await data_gen.create_text_chunks(stream_recap_data.transcript, 0)

text_chunk_batches = await data_gen.generate_text_chunk_batches(text_chunks_no_overlap)

# topic_annotations_str, temp_cost = await data_gen.annotate_major_minor_topics(stream_recap_data.recap)
# cost+=temp_cost
# major_topics, minor_topics = data_gen.process_topic_annotations_str(topic_annotations_str)

responses, annotated_results, temp_cost = await data_gen.annotate_all_batches(text_chunk_batches, topic_annotations_str)
cost+=temp_cost

Cost:  0.023292
Cost:  0.027387
Cost:  0.031020000000000002
Cost:  0.030237
Cost:  0.033393000000000006
Cost:  0.034212
Cost:  0.036552
Cost:  0.036291000000000004
Results:  20
Results:  20
Results:  20
Results:  20
Results:  20
Results:  20
Results:  20
Results:  16


# Proceeding to plot

In [11]:
plot_object, annotated_results, plot_segments, category_locations = await services.stream_plot.process_data(stream_recap_data,  annotated_results, major_topics, minor_topics, video_id)

In [ ]:
plot_object, temp_cost=await services.stream_plot.annotate_extra(video_id, stream_recap_data, plot_object)  

In [12]:
# save plot object to the database
from dataclasses import dataclass, asdict, is_dataclass
stream_recap_data.plot_object=asdict(plot_object)
await sync_to_async(stream_recap_data.save)()

In [ ]:
plot_object.abstractions